In [ ]:
## Recoded for public release 4/26/2024

In [1]:
## Input files
## chr9.corboxscores.corcut.p45.4000.500.txt

## Output files
## chr9.p45.mnts.valleys.tsv
## chr9.p45.4500.mnts.minima.tsv

In [2]:
## Point to the input score file, 
scorepath <- ''
scorefile <- 'chr9.corboxscores.corcut.p45.4000.500.txt'
scorefp <- paste(scorepath, scorefile, sep = '')

In [3]:
## Generic description of what's in the output files
adesc <- 'File 1 is the mountains and valleys file, one line each, File 2 are local minima in large mountains'

In [4]:
## Point to the output files (this code assumes that same output directory)
outpath <- '' 

## Point to the output file 1, 
vspallfile <- 'chr9.p45.mnts.valleys.tsv'
vpfp2 <- paste(outpath, vspallfile, sep = '')

## Point to the output file 2, 
minimafile <- 'chr9.p45.4500.mnts.minima.tsv'
mnma <- paste(outpath, minimafile, sep = '')

In [5]:
## These are variable/parameters used for find mountains and valleys
## Set up threshold and hysteresis parameters
thresh <- 10   ## Valley threshold
hyster_v <- 3  ## Valley hysteresis
hyster_m <- 3  ## Mountain hysteresis

## ## These are variable/parameters used for minima discovery
maxm <- 4500 ## Mountain size, i.e. minimum size of mountain to be broken up
wsize <- 200 ## window (or bin) size used for breaking up mountains
extthresh <- 1000 ## minima threshold (minima above this box score will be rejected)

In [6]:
## Parameters that were used to create the box score file
bx_sz <- 500 ## set this to match the box size used to create the box score file
padval <- -1 ## set this to match the input pad character (start and end score of score_mat)

In [7]:
## Read the score file
score_mat <- read.table(file = scorefp, stringsAsFactors = FALSE, header = FALSE, comment.char = "#")
colnames(score_mat) <- c('filepos_up', 'filepos_down', 'chrpos_up', 'chrpos_down', 'box_score')

## Compute the number of elements in the score file
noeles <- dim(score_mat)[1]

## set up the first and last alleles (elements with valid score)
firstele <- bx_sz
lastele <- noeles - bx_sz + 1

In [8]:
## Force the padded values to very large mountains
score_mat[(score_mat[,5] == padval),5] <- 10000

In [9]:
## Set up code for looping to find valleys and mountains
vsandps <- matrix(as.integer(0), nrow = noeles, ncol = 8)
# colnames(vsandps) <- c('r-type', 'tx_idx')
state <- 'm'
state
Tr1trig <- FALSE
Mr1trig <- FALSE
vpind <- 1
rec2 <- 1
poslast <- 0

[1] "m"

In [10]:
## Asymmetric Hysteresis
## Find the peaks and valleys (psandvs)
for (i in (firstele + 1):(lastele)) {
    if (state == 'm') {
        if (score_mat[i, 5] > (thresh + hyster_m)) Tr1trig <- FALSE
        if ((score_mat[i, 5] < thresh) & !Tr1trig) {
            rec2 <- i
            level <- score_mat[i, 5]
            Tr1trig <- TRUE }
        if (score_mat[i, 5] < (thresh - hyster_v))  {
            state <- 'v'
            vsandps[vpind, 1] <- 'v'
            # vsandps[vpind, 2] <- rec2
            vsandps[vpind, 2:8] <- as.integer(c(rec2, score_mat[rec2, 4],
                                  score_mat[rec2, 3], level, score_mat[rec2, 1],
                                                0, score_mat[rec2, 2]))
            vpind <- vpind + 1
            Tr1trig <- FALSE }}
    else {
        if (score_mat[i, 5] < (thresh - hyster_v)) Mr1trig <- FALSE
        if ((score_mat[i, 5] > thresh) & !Mr1trig) {
            rec2 <- i
            level <- score_mat[i, 5]
            Mr1trig <- TRUE }
        if (score_mat[i, 5] > (thresh + hyster_m))  {
            state <- 'm'
            vsandps[vpind, 1] <- 'm'
            # vsandps[vpind, 2] <- rec2
            vsandps[vpind, 2:8] <- as.integer(c(rec2, score_mat[rec2, 4],
                                    score_mat[rec2, 3], level, score_mat[rec2, 1],
                                                0, score_mat[rec2, 2]))
            vpind <- vpind + 1
            Mr1trig <- FALSE }
        }
    }

In [11]:
## Add a column showing the number of variants for the last region
vsandps[1, 7] <- vsandps[1, 6]
for (i in 2:(vpind - 1)) {
    vsandps[i, 7] <- as.integer(vsandps[i, 6]) - as.integer(vsandps[(i-1), 6])
}
## Most of vsandps is empty. Remove empty rows.
vsandps <- vsandps[1:(vpind-1),]

In [12]:
## Set up the vsandps column names
colnames(vsandps) <- c('TYPE', 'VAR_POS', 'GEN_POS1', 'GEN_POS2',
                       'UNKNOWN1', 'UNKNOWN2', 'NO_OF_VARS', 'FILE_POS')

In [13]:
##### This code commented out for production version (this file format not used)
## Point to the output file (this file is not needed at this point), 
## vsandpsfile <- 'HRC.chr9.corboxscores.noshrink.corcut.p45.2000.500.valleys.mountains.tsv'
## vpfp <- paste(outpath, vsandpsfile, sep ='')

In [14]:
##### This code commented out for production version (this file format not used)
## Write out the appropriate subset of the iterated block, appended   
## write.table(vsandps, file = vpfp, quote = FALSE,
##            sep = '\t', append = FALSE, row.names = FALSE, col.names = TRUE)

In [15]:
# Prepare mountain/valley data extended file format
vspall <- matrix(as.integer(0), nrow = vpind , ncol = 9)
colnames(vspall) <- c('tx-type', 'tx_up', 'pos_up',
                      'tx_dn', 'pos_dn', 'var_est',
                      'vcnt_est', 'pos_est', 'pcnt_est')
vspall <- as.data.frame(vspall)

In [16]:
# Populate extended format of mountain and valley information
vspall[1, 1] <- 'm1'
vspall[1, 2] <- as.integer(0)
vspall[1, 3] <- as.integer(0)
vspall[1, 4] <- score_mat[1, 2]
vspall[1, 5] <- score_mat[1, 4]
vspall[1, 6] <- as.integer(1)
vspall[1, 7] <- as.integer(0)
for (i in 1:(vpind - 1)) {
    # vspall[i + 1, 1] <- vsandps[i, 1]
    if (vsandps[i, 1] =='m') vspall[i + 1, 1] <- paste('m',i, sep='') else vspall[i + 1, 1] <- paste('v',i, sep='')
    vpidx <- as.integer(vsandps[i, 2])
    vspall[i + 1, 2] <- score_mat[vpidx, 1]
    vspall[i + 1, 3] <- score_mat[vpidx, 3]
    vspall[i + 1, 4] <- score_mat[vpidx, 2]
    vspall[i + 1, 5] <- score_mat[vpidx, 4]
    vspall[i + 1, 6] <- as.integer((score_mat[vpidx, 1] + score_mat[vpidx, 2])/2)
    vspall[i + 1, 7] <- as.integer(vspall[i + 1, 6]) - as.integer(vspall[i, 6 ])
    vspall[i + 1, 8] <- as.integer((score_mat[vpidx, 3] + score_mat[vpidx, 4])/2)
    vspall[i + 1, 9] <- as.integer(vspall[i + 1, 8]) - as.integer(vspall[i, 8 ])
    # vspall[i + 1, 10] <- as.integer(vpidx)
}

In [17]:
## Create a more simplied format of moutains and valleys data (format)
vspall2 <- cbind(vspall[1:(vpind -1),],vspall[2:vpind,] )
vspall2 <- vspall2[, c(1,6, 8, 15:18)]
vspall2 <- vspall2[c(1 , 2, 4, 5, 3, 6, 7)]
colnames(vspall2) <- c('reg_type', 'start_var', 'end_var', 'num_vars', 'start_pos', 'end_pos', 'num_nts')
## Even more improved format
vspall3 <- vspall2[, c(1 , 5, 6, 4)]
colnames(vspall3) <- c('TYPE', "START", "STOP", "VARIANTS")

In [18]:
## Write the simplified format of mountains and valleys data, header first
write(file = vpfp2, paste("## ", adesc, sep = ''))
write(file = vpfp2, paste("## Valley, Mountain, Minima Detection, Run on ", Sys.Date(), sep = ''), append = TRUE)
write(file = vpfp2, paste("## Input score file: ", scorefile, sep = ''), append = TRUE)
write(file = vpfp2, paste("## Output File 1 (this file): ", vspallfile, sep = ''), append = TRUE)
write(file = vpfp2, paste("## Output File 2 (companion minima file): ", minimafile, sep = ''), append = TRUE)
write(file = vpfp2, paste("## Number of variants in score file: ", noeles, sep = ''), append = TRUE)
write(file = vpfp2, paste("## Box size used in creating input file: ", bx_sz, sep = ''), append = TRUE)
write(file = vpfp2, paste("## Valley Threshold (box score): ", thresh, sep = ''), append = TRUE)
write(file = vpfp2, paste("## Valley Hysteresis: ", hyster_v, sep = ''), append = TRUE)
write(file = vpfp2, paste("## Mountain Hysteresis: ", hyster_m, sep = ''), append = TRUE)
write(file = vpfp2, paste("## No. Mountains/Valleys Detected: ", dim(vspall3)[1], sep = ''), append = TRUE)
write(file = vpfp2, paste("## Column Titles: ", sep = ''), append = TRUE)
coltitles <- c('TYPE, START, STOP, VARIANTS')
write(file = vpfp2, paste("## ", coltitles, sep = ''), append = TRUE)

In [19]:
## Write the simplified format of mountains and valleys data, data second
## Write out the appropriate subset of the iterated block, appended to header  
write.table(vspall3, file = vpfp2, quote = FALSE,
            sep = '\t', append = TRUE, row.names = FALSE, col.names = FALSE)

In [20]:
## Set up loop variable for minima discovery
fcnt <- 0
fmax <- 100000000 ## Artificial limit, can be reduced for debugging
minouts <- NULL
for (i in 1:(dim(vspall)[1]-1)) {
    vspi <- vspall[i,]
    if (((vspall[i + 1, 4]) - vspall[i, 2]) > maxm) {
        fcnt <- fcnt + 1
        if (fcnt > fmax) break
        p2 <- vspall[i + 1, 4]
        p1 <- vspall[i, 2]
        p1p <- which(score_mat[,2] >= p1)[1]
        p2p <- which(score_mat[,2] >= p2)[1]
        testv <- score_mat[p1p:p2p, 5]
        its <- 1 + as.integer(length(testv)/wsize)
        testmin <- rep(0, times = its)
        regstart <- rep(0, times = its)
        for (j in 1:its) {
            regstart[j] <- (j - 1) * wsize + 1
            wmin <- min(testv[((j - 1) * wsize + 1):(j * wsize)])   
            testmin[j] <- which(testv[((j - 1) * wsize + 1):(j * wsize)] == wmin)[1] + (j - 1) * wsize
            }
        skip1st <- 2:(length(testmin) - 1)
        testmin <- testmin[skip1st]
        testcut <- testmin[testv[testmin] < extthresh]
        if (length(testcut) > 0) {
            minouts <- rbind(minouts, cbind(paste(vspi[1,1],'.',1:length(testcut), sep =''),
                        as.integer((score_mat[(testcut + p1p - 1), 3] + score_mat[(testcut + p1p - 1), 4])/2),
                        score_mat[(testcut + p1p - 1), 2],              
                        rep(score_mat[(p1p), 2], times = length(testcut)),
                        score_mat[(testcut + p1p - 1),5]))
            }
        }
    }

In [21]:
## Repackage output, compute inter minima variant count (from nears upstream minima)
minouts2 <- minouts
minouts2 <- cbind(minouts2, as.integer(minouts2[,3]) - as.integer(minouts2[,4]), as.integer(minouts2[,3]))
minouts2[1, 7] <- minouts2[1, 6]
j <- 1
for (i in 2:dim(minouts2)[1]) {
    if (minouts2[i, 4] != minouts2[i - 1, 4]) {
        j <- i
        minouts2[i, 7] <- as.integer(minouts2[i, 3]) - as.integer(minouts2[i, 4])}
    else { minouts2[i, 7] <- as.integer(minouts2[i, 3]) - 
          as.integer(minouts2[i, 4]) - sum(as.integer(minouts2[j:(i - 1), 7]))}
}

In [22]:
## Set up the minouts variable for writing to a file (Intermediate format)
colnames(minouts) <- c('SUBTYE', 'POSITION', 'VARIANT_POS', 'MOUNT_START_V', 'MINIMA_SCORE')
# head(minouts)

In [23]:
##### This code commented out for production version (this file format not used)
## Point to the output minima file (raw version)
## minfile <- 'HRC.chr9.corboxscores.noshrink.corcut.p45.2000.500.minouts.tsv'
## minfp <- paste(scorepath, minfile, sep ='')

In [24]:
##### This code commented out for production version (this file format not used)
## Write out the appropriate raw minima file   
## write.table(minouts, file = minfp, quote = FALSE,
##            sep = '\t', append = FALSE, row.names = FALSE, col.names = TRUE)

In [25]:
## Repackage output again, with just essential columns and proper column names 
minouts3 <- minouts2[, c(1, 2, 7, 5)]
colnames(minouts3) <- c("SUBTYPE", "POSITION", "VARIANTS", "BOXSCORE")

In [26]:
## Filter out minima that had NA in scoring (had less than 200 common variants to meet wsize)
## Filter out small regions of minima (variants less than wsize)
minouts3 <- minouts3[!(is.na(minouts3[,2]) | is.na(minouts3[,3]) | is.na(minouts3[,4])), ]
minouts3 <- minouts3[!(as.integer(minouts3[,3]) <= wsize), ]

In [27]:
write(file = mnma, paste("## ", adesc, sep = ''))
write(file = mnma, paste("## Valley, Mountain, Minima Detection, Run on ", Sys.Date(), sep = ''), append = TRUE)
write(file = mnma, paste("## Input score file: ", scorefile, sep = ''), append = TRUE)
write(file = mnma, paste("## Output File 1 (companion file): ", vspallfile, sep = ''), append = TRUE)
write(file = mnma, paste("## Output File 2 (this file): ", minimafile, sep = ''), append = TRUE)
write(file = mnma, paste("## Number of variants in score file: ", noeles, sep = ''), append = TRUE)
write(file = mnma, paste("## Minimum Mountain to Find Minima: ", maxm, sep = ''), append = TRUE)
write(file = mnma, paste("## Window Size (Variants) to Score Minima: ", wsize, sep = ''), append = TRUE)
write(file = mnma, paste("## Highest Allowed Minima (Box Score): ", extthresh, sep = ''), append = TRUE)
write(file = mnma, paste("## Total Number of new Minima Found: ", dim(minouts3)[1], sep = ''), append = TRUE)
write(file = mnma, paste("## Column Titles: ", sep = ''), append = TRUE)
coltitles <- c('SUBTYPE, POSITION, VARIANTS, BOXSCORE')
write(file = mnma, paste("## ", coltitles, sep = ''), append = TRUE)

In [28]:
## Write out the appropriate subset of the iterated block, appended   
write.table(minouts3, file = mnma, quote = FALSE,
            sep = '\t', append = TRUE, row.names = FALSE, col.names = FALSE)